# Task 2 - Timely stream processing
In this task, you have to help a bank security department. They need to find suspicious transfers. The first idea to find these types of events is to look at the transfers sent shortly after app activation. Help them and write the proper job!

In [ ]:
%reload_ext streaming_jupyter_integrations.magics
%flink_connect

You have access to 2 Kafka topics. In the topic `transfers` you have data about transfers. In the topic `app_activations` you can find information about app activations. Sample messages are available in `data/` directory. First, you have to prepare definitions for these tables. You can find Kafka Table example [here](https://nightlies.apache.org/flink/flink-docs-master/docs/connectors/table/kafka/).

In [ ]:
%%flink_execute_sql
CREATE TABLE transfers (
    transaction_id STRING,
    user_id INT,
    amount DECIMAL(38,2),
    currency STRING,
    IBAN STRING,
    ts TIMESTAMP(3),
    WATERMARK FOR ts AS ts
) WITH (
    'connector' = 'kafka',
    'topic' = 'transfers',
    'properties.bootstrap.servers' = '172.18.0.3:9092',
    'properties.group.id' = 'task2',
    'scan.startup.mode' = 'earliest-offset',
    'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE app_activations (
    user_id INT,
    device_fingerprint STRING,
    app_version STRING,
    ts TIMESTAMP(3),
    WATERMARK FOR ts AS ts
) WITH (
    'connector' = 'kafka',
    'topic' = 'app_activations',
    'properties.bootstrap.servers' = '172.18.0.3:9092',
    'properties.group.id' = 'task2',
    'scan.startup.mode' = 'earliest-offset',
    'format' = 'json'
)

Now it's time to write a select clause. Find the transfers happened max 2 minutes after app activation.

In [ ]:
%%flink_execute_sql
SELECT
    t.*
FROM
    app_activations a,
    transfers t 
WHERE
    a.user_id = t.user_id
AND t.ts BETWEEN a.ts AND a.ts + INTERVAL '2' MINUTES

## Windows&aggregations
Director of the security department has one extra request. He would like to know after every hour how many suspicious transfert happened THIS DAY. Please help him. More about windows you can find [HERE](https://nightlies.apache.org/flink/flink-docs-master/docs/dev/table/sql/queries/window-tvf/).

In [ ]:
%%flink_execute_sql
WITH suspicious AS (
    SELECT
        t.*
    FROM
        app_activations a,
        transfers t 
    WHERE a.user_id = t.user_id
    AND t.ts BETWEEN a.ts AND a.ts + INTERVAL '2' MINUTES
)
SELECT
    window_start,
    window_end,
    count(distinct transaction_id) as t_cnt
FROM TABLE(
    CUMULATE(
        TABLE suspicious,
        DESCRIPTOR(ts),
        INTERVAL '1' HOUR,
        INTERVAL '1' DAY))
GROUP BY
    window_start,
    window_end